<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Friedmann-equations" data-toc-modified-id="Friedmann-equations-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Friedmann equations</a></span></li></ul></div>

# Friedmann equations

This worksheet demonstrates a few capabilities of [SageManifolds](http://sagemanifolds.obspm.fr) (version 1.0, as included in SageMath 7.5) in computations regarding cosmological spacetimes with Friedmann-Lemaître-Robertson-Walker (FLRW) metrics.

Click [here](https://raw.githubusercontent.com/sagemanifolds/SageManifolds/master/Worksheets/v1.0/SM_Friedmann_equations.ipynb) to download the worksheet file (ipynb format). To run it, you must start SageMath within the Jupyter notebook, via the command `sage -n jupyter`

*NB:* a version of SageMath at least equal to 7.5 is required to run this worksheet:

In [1]:
version()

'SageMath version 8.0, Release Date: 2017-07-21'

First we set up the notebook to display mathematical objects using LaTeX formatting:

In [2]:
%display latex

We declare the spacetime M as a 4-dimensional manifold:

In [3]:
M = Manifold(4, 'M')
print(M)

4-dimensional differentiable manifold M


We introduce the standard FLRW coordinates, via the method `chart()`, the argument of which is a string expressing the coordinates names, their ranges (the default is $(-\infty,+\infty)$) and their LaTeX symbols:

In [4]:
fr.<t,r,th,ph> = M.chart(r't r:[0,+oo) th:[0,pi]:\theta ph:[0,2*pi):\phi')
fr

Chart (M, (t, r, th, ph))

<p>Assuming that the speed of light c=1, let us define a few variables: Newton's constant $G$, the cosmological constant $\Lambda$, the spatial curvature constant $k$, the scale factor $a(t)$, the fluid proper density $\rho(t)$ and the fluid pressure $p(t)$:</p>

In [5]:
var('G, Lambda, k', domain='real')
a = M.scalar_field(function('a')(t), name='a')
rho = M.scalar_field(function('rho')(t), name='rho')
p = M.scalar_field(function('p')(t), name='p')

The FLRW metric is defined by its components in the manifold's default frame, i.e. the frame associated with the FLRW coordinates:

In [6]:
g = M.lorentzian_metric('g')
g[0,0] = -1
g[1,1] = a*a/(1 - k*r^2)
g[2,2] = a*a*r^2
g[3,3] = a*a*(r*sin(th))^2
g.display()

g = -dt*dt - a(t)^2/(k*r^2 - 1) dr*dr + r^2*a(t)^2 dth*dth + r^2*a(t)^2*sin(th)^2 dph*dph

<p>A matrix view of the metric components:</p>

In [7]:
g[:]

[                  -1                    0                    0                    0]
[                   0  -a(t)^2/(k*r^2 - 1)                    0                    0]
[                   0                    0           r^2*a(t)^2                    0]
[                   0                    0                    0 r^2*a(t)^2*sin(th)^2]

<p>The Levi-Civita connection associated with the metric is computed:</p>

In [8]:
nabla = g.connection()
g.christoffel_symbols_display()

Gam^t_r,r = -a(t)*d(a)/dt/(k*r^2 - 1) 
Gam^t_th,th = r^2*a(t)*d(a)/dt 
Gam^t_ph,ph = r^2*a(t)*sin(th)^2*d(a)/dt 
Gam^r_t,r = d(a)/dt/a(t) 
Gam^r_r,r = -k*r/(k*r^2 - 1) 
Gam^r_th,th = k*r^3 - r 
Gam^r_ph,ph = (k*r^3 - r)*sin(th)^2 
Gam^th_t,th = d(a)/dt/a(t) 
Gam^th_r,th = 1/r 
Gam^th_ph,ph = -cos(th)*sin(th) 
Gam^ph_t,ph = d(a)/dt/a(t) 
Gam^ph_r,ph = 1/r 
Gam^ph_th,ph = cos(th)/sin(th)

Ricci tensor:

In [9]:
Ricci = nabla.ricci()
Ricci.display()

Ric(g) = -3*d^2(a)/dt^2/a(t) dt*dt - (2*(d(a)/dt)^2 + a(t)*d^2(a)/dt^2 + 2*k)/(k*r^2 - 1) dr*dr + (2*r^2*(d(a)/dt)^2 + r^2*a(t)*d^2(a)/dt^2 + 2*k*r^2) dth*dth + (2*r^2*(d(a)/dt)^2 + r^2*a(t)*d^2(a)/dt^2 + 2*k*r^2)*sin(th)^2 dph*dph

<p>Ricci scalar ($R^\mu_{\ \, \mu}$):</p>

In [10]:
Ricci_scalar = g.ricci_scalar()
Ricci_scalar.display()

r(g): M --> R
   (t, r, th, ph) |--> 6*((d(a)/dt)^2 + a(t)*d^2(a)/dt^2 + k)/a(t)^2

<p>The fluid 4-velocity:</p>

In [11]:
u = M.vector_field('u')
u[0] = 1
u.display()

u = d/dt

In [12]:
g(u,u).expr()

-1

<p>Perfect fluid energy-momentum tensor $T$:</p>

In [13]:
u_form = u.down(g) # the 1-form associated to u by metric duality
T = (rho+p)*(u_form*u_form) + p*g
T.set_name('T')
print(T)
T.display()

Field of symmetric bilinear forms T on the 4-dimensional differentiable manifold M


T = rho(t) dt*dt - a(t)^2*p(t)/(k*r^2 - 1) dr*dr + r^2*a(t)^2*p(t) dth*dth + r^2*a(t)^2*p(t)*sin(th)^2 dph*dph

<p>The trace of $T$ (we use index notation to denote the double contraction $g^{ab} T_{ab}$):</p>

In [14]:
Ttrace = g.inverse()['^ab']*T['_ab']
Ttrace.display()

M --> R
(t, r, th, ph) |--> 3*p(t) - rho(t)

<p>Einstein equation: $R_{\mu \nu} - {1 \over 2} R g_{\mu \nu} + \Lambda g_{\mu \nu} = {8 \pi G} T_{\mu \nu}$</p>

In [15]:
E1 = Ricci - Ricci_scalar/2*g + Lambda*g - (8*pi*G)*T
print("First Friedmann equation:\n")
E1[0,0].expr().expand() == 0

First Friedmann equation:



-8*pi*G*rho(t) - Lambda + 3*diff(a(t), t)^2/a(t)^2 + 3*k/a(t)^2 == 0

<p>Trace-reversed version of the Einstein equation: $R_{\mu \nu} - \Lambda g_{\mu \nu} = {8 \pi G} \left(T_{\mu \nu} - {1 \over 2}T\,g_{\mu \nu}\right)$</p>

In [16]:
E2 = Ricci - Lambda*g - (8*pi*G)*(T - Ttrace/2*g)
print("Second Friedmann equation:\n")
E2[0,0].expr().expand() == 0

Second Friedmann equation:



-12*pi*G*p(t) - 4*pi*G*rho(t) + Lambda - 3*diff(a(t), t, t)/a(t) == 0